# <font color='darkblue'>Setup</font> 

Todo:

Add in the threshold method to compare

## <font color='orange'>Packages</font> 

In [31]:
# Packages -----------------------------------------------#

# Data Analysis
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats
from scipy.ndimage import gaussian_filter
import datetime as dt
from scipy.interpolate import griddata

# Plotting
import cartopy.crs as ccrs
from cartopy.feature import LAND
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Timing Processes and Progress
import time
from tqdm import tqdm

# make sure the figures plot inline rather than at the end
%matplotlib inline

## <font color='orange'>Functions</font> 

In [32]:

# put coastlines on map and set xy limits ---------------------------------------------#
def add_land(ax,bounds= [35,120,-20,30]):
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
    ax.add_feature(cfeature.LAND,color='gray',zorder = 1)
    ax.background_patch.set_facecolor('k')
    ax.coastlines(resolution='110m',zorder = 2)
    g = ax.gridlines(draw_labels=True,alpha=0)
    g.xlabels_top = False
    g.ylabels_right = False
    g.xlabel_style = {'size': 15}
    g.ylabel_style = {'size': 15}
    g.xformatter = LONGITUDE_FORMATTER
    g.yformatter = LATITUDE_FORMATTER
    ax.axes.axis('tight')
    ax.set_extent(bounds, crs=ccrs.PlateCarree())
    return None 

# mask coastlines ---------------------------------------------------------------------#
def mask_coast(c_lon,c_lat,bounds):
    data=xr.open_dataset('/tigress/GEOCLIM/LRGROUP/shared_data/pco2_flux_coastal_Roobaert/mask_ocean.nc')
    mask_coast=np.array(data.mask_coastal2).astype(int).T
    lat=np.array(data.latitude)
    lon=np.array(data.longitude)

    mask_lon=np.logical_and(lon>bounds[0],lon<bounds[1])
    mask_lat=np.logical_and(lat>bounds[2],lat<bounds[3])

    lon=lon[mask_lon]
    lat=lat[mask_lat]

    mask_coast=mask_coast[mask_lat]
    mask_coast=mask_coast[:,mask_lon]

    lonlon,latlat=np.meshgrid(lon,lat)

    # what are these for? I should probably fix it
    lon_dot=np.array([70,70])
    lat_dot=np.array([10,19.5])

    mask=[]
    for lo,la in zip(c_lon,c_lat):
        if len(lon[lon<=lo])>0 and len(lat[lat>=la])>0 and len(lon[lon>=lo])>0 and len(lat[lat<=la])>0:
            lon_lim=[lon[lon<=lo][-1],lon[lon>=lo][0]]
            lat_lim=[lat[lat<=la][-1],lat[lat>=la][0]]
            mask_lon=np.logical_or(lon==lon_lim[0],lon==lon_lim[1])
            mask_lat=np.logical_or(lat==lat_lim[0],lat==lat_lim[1])
            mask_tmp=mask_coast[mask_lat]
            mask_tmp=mask_tmp[:,mask_lon]
            mask.append(np.mean(mask_tmp)>0)
        else:
            mask.append(False)
    mask=np.array(mask)
    return mask

# create pcolormesh lat/lon vals ------------------------------------------------------#
def pcolor_xy(x,y):
    # extend longitude by 2
    x_extend = np.zeros(x.size+2)
    # fill in internal values
    x_extend[1:-1] = x # fill up with original values
    # fill in extra endpoints
    x_extend[0] = x[0]-np.diff(x)[0]
    x_extend[-1] = x[-1]+np.diff(x)[-1]
    # calculate the midpoints
    x_pcolormesh_midpoints = x_extend[:-1]+0.5*(np.diff(x_extend))

    # extend latitude by 2
    y_extend = np.zeros(y.size+2)
    # fill in internal values
    y_extend[1:-1] = y
    # fill in extra endpoints
    y_extend[0] = y[0]-np.diff(y)[0]
    y_extend[-1] = y[-1]+np.diff(y)[-1]
    # calculate the midpoints
    y_pcolormesh_midpoints = y_extend[:-1]+0.5*(np.diff(y_extend))
    
    return x_pcolormesh_midpoints,y_pcolormesh_midpoints

# month of min OCD plus other vars  -----------------------------------------------------#
def latlonbin_min_OCD(OCD,TCD,doxy_upper,doxy_lower,lat,lon,bounds,binwidth):
    import numpy as np
    
    # create a pandas dataframe
    df = pd.DataFrame(dict(
            doxy_upper = np.array(doxy_upper),
            doxy_lower = np.array(doxy_lower),
            OCD = np.array(OCD),
            TCD = np.array(TCD),
            lat= np.array(lat),
            lon= np.array(lon),
            month = np.array(OCD.time.dt.month)
        ))
    
    # create 1 degree bins
    latedges = np.arange(bounds[2]-(binwidth/2),bounds[3]+(binwidth/2),binwidth)
    lat_inds = list(range(len(latedges)-1))

    lonedges = np.arange(bounds[0]-(binwidth/2),bounds[1]+(binwidth/2),binwidth)
    lon_inds = list(range(len(lonedges)-1))

    latbins = latedges[1:]-(binwidth/2)
    lonbins = lonedges[1:]-(binwidth/2)

    df['latedges'] = pd.cut(lat, latedges)
    df['lonedges'] = pd.cut(lon, lonedges)
    df['latbins_ind'] = pd.cut(lat, latedges,labels = lat_inds)
    df['lonbins_ind'] = pd.cut(lon, lonedges,labels = lon_inds)
    df['lat_lon_indx']=df.groupby(['latbins_ind', 'lonbins_ind']).ngroup()
    grouped = df.groupby(['latbins_ind', 'lonbins_ind'])

    min_OCD = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_OCD_month = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_OCD_TCD = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_OCD_doxy_upper = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_OCD_doxy_lower = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan

    #extract the data for each group
    for name, group in grouped:
        i = np.array(group.latbins_ind)
        j = np.array(group.lonbins_ind)
        gpmonth = np.array(group.month)
        gpOCD = np.array(group.OCD)
        gpTCD = np.array(group.TCD)
        gpdoxy_upper = np.array(group.doxy_upper)
        gpdoxy_lower = np.array(group.doxy_lower)

        # find month of minimum OCD
        if ~np.isnan(np.nanmin(gpOCD)):
            
            min_OCD[i[0],j[0]] = np.nanmin(gpOCD)
            
            ind = np.nanargmin(gpOCD)
            
            min_OCD_month[i[0],j[0]] = gpmonth[ind]
            min_OCD_TCD[i[0],j[0]] = gpTCD[ind]       
            min_OCD_doxy_upper[i[0],j[0]] = gpdoxy_upper[ind]
            min_OCD_doxy_lower[i[0],j[0]] = gpdoxy_lower[ind]

    return(np.array(min_OCD,dtype = float),np.array(min_OCD_month,dtype = float),
           np.array(min_OCD_TCD,dtype = float),
           np.array(min_OCD_doxy_upper,dtype = float),
           np.array(min_OCD_doxy_lower,dtype = float),
           latbins,lonbins)

# month of min doxy plus other vars  -----------------------------------------------------#
def latlonbin_min_doxy(doxy,OCD,TCD,lat,lon,bounds,binwidth):
    import numpy as np
    
    if len(doxy.shape) > 1:
        doxy = np.nanmin(doxy,1)
    
    # create a pandas dataframe
    df = pd.DataFrame(dict(
            doxy = np.array(doxy),
            OCD = np.array(OCD),
            TCD = np.array(TCD),
            lat= np.array(lat),
            lon= np.array(lon),
            month = np.array(OCD.time.dt.month)
        ))
    
    # create 1 degree bins
    latedges = np.arange(bounds[2]-(binwidth/2),bounds[3]+(binwidth/2),binwidth)
    lat_inds = list(range(len(latedges)-1))

    lonedges = np.arange(bounds[0]-(binwidth/2),bounds[1]+(binwidth/2),binwidth)
    lon_inds = list(range(len(lonedges)-1))

    latbins = latedges[1:]-(binwidth/2)
    lonbins = lonedges[1:]-(binwidth/2)

    df['latedges'] = pd.cut(lat, latedges)
    df['lonedges'] = pd.cut(lon, lonedges)
    df['latbins_ind'] = pd.cut(lat, latedges,labels = lat_inds)
    df['lonbins_ind'] = pd.cut(lon, lonedges,labels = lon_inds)
    df['lat_lon_indx']=df.groupby(['latbins_ind', 'lonbins_ind']).ngroup()
    grouped = df.groupby(['latbins_ind', 'lonbins_ind'])

    min_doxy = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_doxy_month = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_doxy_TCD = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan
    min_doxy_OCD = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray) * np.nan

    #extract the data for each group
    for name, group in grouped:
        i = np.array(group.latbins_ind)
        j = np.array(group.lonbins_ind)
        gpmonth = np.array(group.month)
        gpOCD = np.array(group.OCD)
        gpTCD = np.array(group.TCD)
        gpdoxy = np.array(group.doxy)

        # find month of minimum OCD
        if ~np.isnan(np.nanmin(gpdoxy)):
            
            min_doxy[i[0],j[0]] = np.nanmin(gpdoxy)
            
            ind = np.nanargmin(gpdoxy)
            
            min_doxy_month[i[0],j[0]] = gpmonth[ind]
            min_doxy_TCD[i[0],j[0]] = gpTCD[ind]  
            min_doxy_OCD[i[0],j[0]] = gpOCD[ind]    

    return(np.array(min_doxy,dtype = float),np.array(min_doxy_month,dtype = float),
           np.array(min_doxy_TCD,dtype = float),
           np.array(min_doxy_OCD,dtype = float),
           latbins,lonbins)

## <font color='orange'>Read Data</font> 

# <font color='darkblue'>Month of Minimum Oxygen/OCD</font> 

## <font color='orange'>Month of Minimum Oxygen</font> 

In [ ]:
# min_doxy,min_doxy_month,min_doxy_TCD,min_doxy_OCD,latbins,lonbins = latlonbin_min_doxy(np.nanmean(ds_AWG.doxy,1),
#                                                                                       ds_AWG.OCD,ds_AWG.TCD,
#                                                                                       ds_AWG.lat,ds_AWG.lon,
#                                                                                       bounds,binwidth)
# min_doxy_upper,min_doxy_upper_month,min_doxy_upper_TCD,min_doxy_upper_OCD,_,_ = latlonbin_min_doxy(ds_AWG.doxy_upper_mean,
#                                                                                       ds_AWG.OCD,ds_AWG.TCD,
#                                                                                       ds_AWG.lat,ds_AWG.lon,
#                                                                                       bounds,binwidth)

# min_doxy_lower,min_doxy_lower_month,min_doxy_lower_TCD,min_doxy_lower_OCD,_,_ = latlonbin_min_doxy(ds_AWG.doxy_lower_mean,
#                                                                                       ds_AWG.OCD,ds_AWG.TCD,
#                                                                                       ds_AWG.lat,ds_AWG.lon,
#                                                                                       bounds,binwidth)                                                                                                             

# min_doxy_TC,min_doxy_TC_month,min_doxy_TC_TCD,min_doxy_TC_OCD,_,_ = latlonbin_min_doxy(ds_AWG.doxy_TC_mean,
#                                                                                       ds_AWG.OCD,ds_AWG.TCD,
#                                                                                       ds_AWG.lat,ds_AWG.lon,
#                                                                                       bounds,binwidth)       

# # add to dataset
# ds_AWG['min_doxy'] = xr.DataArray(min_doxy,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_month'] = xr.DataArray(min_doxy_month,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_TCD'] = xr.DataArray(min_doxy_TCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_OCD'] = xr.DataArray(min_doxy_OCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])

# ds_AWG['min_doxy_upper'] = xr.DataArray(min_doxy_upper,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_upper_month'] = xr.DataArray(min_doxy_upper_month,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_upper_TCD'] = xr.DataArray(min_doxy_upper_TCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_upper_OCD'] = xr.DataArray(min_doxy_upper_OCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])

# ds_AWG['min_doxy_lower'] = xr.DataArray(min_doxy_lower,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_lower_month'] = xr.DataArray(min_doxy_lower_month,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_lower_TCD'] = xr.DataArray(min_doxy_lower_TCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_lower_OCD'] = xr.DataArray(min_doxy_lower_OCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])

# ds_AWG['min_doxy_TC'] = xr.DataArray(min_doxy_TC,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_TC_month'] = xr.DataArray(min_doxy_TC_month,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_TC_TCD'] = xr.DataArray(min_doxy_TC_TCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_doxy_TC_OCD'] = xr.DataArray(min_doxy_TC_OCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])

In [ ]:
# # Plotting
# xx,yy = np.meshgrid(ds_AWG.lonbins,ds_AWG.latbins)
# xx = xx.flatten()
# yy = yy.flatten()

# bounds = [35,120,-20,30]
# s = 20

# fig, axes = plt.subplots(nrows = 1, ncols = 2,figsize=(20,4),subplot_kw={'projection': ccrs.PlateCarree()})


# # # total o2

# # # limits
# # cmin = 50
# # cmax = 200
# # cmap = plt.cm.Spectral
# # p = axes[0,0].scatter(xx,yy,c = ds_AWG.min_doxy.stack(z=('latbins','lonbins')),marker='s',
# #                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# # add_land(axes[0,0],bounds)
# # axes[0,0].set_extent(bounds, crs=ccrs.PlateCarree())
# # axes[0,0].set_title('Minimum Mean [O2] Total Water Column')
# # cbar = fig.colorbar(p,ax=axes[0,0], pad=0.04)
# # cbar.set_label('Dissolved Oxygen ($units$)')

# # # limits
# # cmin = 1
# # cmax = 12
# # cmap = plt.cm.twilight_shifted
# # p2 = axes[0,1].scatter(xx,yy,c = ds_AWG.min_doxy_month.stack(z=('latbins','lonbins')),marker='s',
# #                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# # add_land(axes[0,1],bounds)
# # axes[0,1].set_title('Month of Minimum [O2]')
# # cbar = fig.colorbar(p2,ax=axes[0,1], pad=0.04)
# # cbar.set_ticks(np.linspace(0,12,13))
# # cbar.set_ticklabels(['dec','jan','feb','mar','apr','may','jun',
# #                     'jul','aug','sep','oct','nov','dec'])
# # cbar.set_label('month')


# # # upper 50 dbar

# # # limits
# # cmin = 50
# # cmax = 200
# # cmap = plt.cm.Spectral
# # p3 = axes[1,0].scatter(xx,yy,c = ds_AWG.min_doxy_upper.stack(z=('latbins','lonbins')),marker='s',
# #                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# # add_land(axes[1,0],bounds)
# # axes[1,0].set_extent(bounds, crs=ccrs.PlateCarree())
# # axes[1,0].set_title('Minimum Mean [O2] 0-50 dbar')
# # cbar = fig.colorbar(p3,ax=axes[1,0], pad=0.04)
# # cbar.set_label('Dissolved Oxygen ($units$)')

# # cmin = 1
# # cmax = 12
# # cmap = plt.cm.twilight_shifted
# # p4 = axes[1,1].scatter(xx,yy,c = ds_AWG.min_doxy_upper_month.stack(z=('latbins','lonbins')),marker='s',
# #                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# # add_land(axes[1,1],bounds)
# # axes[1,1].set_title('Month of Minimum [O2] 0-50 dbar')
# # cbar = fig.colorbar(p4,ax=axes[1,1], pad=0.04)
# # cbar.set_ticks(np.linspace(0,12,13))
# # cbar.set_ticklabels(['dec','jan','feb','mar','apr','may','jun',
# #                     'jul','aug','sep','oct','nov','dec'])
# # cbar.set_label('month')


# # # 50-200 dbar

# # # limits
# # cmin = 50
# # cmax = 200
# # cmap = plt.cm.Spectral
# # p5 = axes[2,0].scatter(xx,yy,c = ds_AWG.min_doxy_lower.stack(z=('latbins','lonbins')),marker='s',
# #                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# # add_land(axes[2,0],bounds)
# # axes[2,0].set_extent(bounds, crs=ccrs.PlateCarree())
# # axes[2,0].set_title('Minimum Mean [O2] 50-200 dbar')
# # cbar = fig.colorbar(p5,ax=axes[2,0], pad=0.04)
# # cbar.set_label('Dissolved Oxygen ($units$)')

# # cmin = 1
# # cmax = 12
# # cmap = plt.cm.twilight_shifted
# # p6 = axes[2,1].scatter(xx,yy,c = ds_AWG.min_doxy_lower_month.stack(z=('latbins','lonbins')),marker='s',
# #                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# # add_land(axes[2,1],bounds)
# # axes[2,1].set_title('Month of Minimum [O2] 50-200 dbar')
# # cbar = fig.colorbar(p6,ax=axes[2,1], pad=0.04)
# # cbar.set_ticks(np.linspace(0,12,13))
# # cbar.set_ticklabels(['dec','jan','feb','mar','apr','may','jun',
# #                     'jul','aug','sep','oct','nov','dec'])
# # cbar.set_label('month')

# # 0-200 dbar

# # limits
# cmin = 50
# cmax = 200
# cmap = plt.cm.Spectral
# p7 = axes[0].scatter(xx,yy,c = ds_AWG.min_doxy_TC.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[0],bounds)
# axes[0].set_extent(bounds, crs=ccrs.PlateCarree())
# axes[0].set_title('Minimum Mean [O2] 0-200 dbar')
# cbar = fig.colorbar(p7,ax=axes[0], pad=0.04)
# cbar.set_label('Dissolved Oxygen ($units$)')

# cmin = 1
# cmax = 12
# cmap = plt.cm.twilight_shifted
# p8 = axes[1].scatter(xx,yy,c = ds_AWG.min_doxy_TC_month.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[1],bounds)
# axes[1].set_title('Month of Minimum [O2] 0-200 dbar')
# cbar = fig.colorbar(p8,ax=axes[1], pad=0.04)
# cbar.set_ticks(np.linspace(0,12,13))
# cbar.set_ticklabels(['dec','jan','feb','mar','apr','may','jun',
#                     'jul','aug','sep','oct','nov','dec'])
# cbar.set_label('month')



## <font color='orange'>Month of Minimum OCD</font> 

In [ ]:
# min_OCD,min_OCD_month,min_OCD_TCD,min_OCD_doxy_upper,min_OCD_doxy_lower,latbins,lonbins = latlonbin_min_OCD(ds_AWG.OCD,ds_AWG.TCD,
#                                                                                            doxy_upper,doxy_lower,
#                                                                                            ds_AWG.lat,ds_AWG.lon,
#                                                                                            bounds,binwidth)

# # do smoothed version too


# # add to dataset
# ds_AWG['min_OCD'] = xr.DataArray(min_OCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_month'] = xr.DataArray(min_OCD_month,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_TCD'] = xr.DataArray(min_OCD_TCD,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_doxy_upper'] = xr.DataArray(min_OCD_doxy_upper,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_doxy_lower'] = xr.DataArray(min_OCD_doxy_lower,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])

In [ ]:
# # Plotting
# xx,yy = np.meshgrid(ds_AWG.lonbins,ds_AWG.latbins)
# xx = xx.flatten()
# yy = yy.flatten()

# bounds = [35,120,-20,30]
# s = 20

# fig, axes = plt.subplots(nrows = 2, ncols = 2,figsize=(20,12),subplot_kw={'projection': ccrs.PlateCarree()})

# # limits
# cmin = 20
# cmax = 160
# cmap = plt.cm.Spectral
# p = axes[0,0].scatter(xx,yy,c = ds_AWG.min_OCD.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[0,0],bounds)
# axes[0,0].set_extent(bounds, crs=ccrs.PlateCarree())
# axes[0,0].set_title('Minimum OCD')
# cbar = fig.colorbar(p,ax=axes[0,0], pad=0.04)
# cbar.set_label('Pressure ($dbar$)')

# # limits
# cmin = 1
# cmax = 12
# cmap = plt.cm.Spectral
# p2 = axes[0,1].scatter(xx,yy,c = ds_AWG.min_OCD_month.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[0,1],bounds)
# axes[0,1].set_title('Month of Minimum OCD')
# cbar = fig.colorbar(p2,ax=axes[0,1], pad=0.04)
# cbar.set_label('month')

# # limits
# cmin = 0
# cmax = 200
# cmap = plt.cm.Spectral
# p3 = axes[1,0].scatter(xx,yy,c = ds_AWG.min_OCD_doxy_upper.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[1,0],bounds)
# axes[1,0].set_extent(bounds, crs=ccrs.PlateCarree())
# axes[1,0].set_title('[O2] 0-50 dbar')
# cbar = fig.colorbar(p3,ax=axes[1,0], pad=0.04)
# cbar.set_label('Dissolved Oxygen ($units$)')

# cmin = 0
# cmax = 200
# cmap = plt.cm.Spectral
# p4 = axes[1,1].scatter(xx,yy,c = ds_AWG.min_OCD_doxy_lower.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[1,1],bounds)
# axes[1,1].set_title('[O2] 50-200 dbar')
# cbar = fig.colorbar(p4,ax=axes[1,1], pad=0.04)
# cbar.set_label('Dissolved Oxygen ($units$)')

## <font color='orange'>Month of Minimum OCD Anomaly</font> 

In [ ]:
# # find average for upper 0-50 dbar
# doxy_upper = np.array(ds_AWG.doxy[:,ds_AWG.pres<50])
# doxy_upper = np.nanmean(doxy_upper,1)

# # find average for lower 55-200 dbar 
# doxy_lower = np.array(ds_AWG.doxy[:,(ds_AWG.pres>50) & (ds_AWG.pres<200)])
# doxy_lower = np.nanmean(doxy_lower,1)

# min_OCD_anom,min_OCD_anom_month,min_OCD_anom_TCD_anom,min_OCD_anom_doxy_upper,min_OCD_anom_doxy_lower,latbins,lonbins = latlonbin_min_OCD(ds_AWG.OCD_anom,ds_AWG.TCD_anom,
#                                                                                            doxy_upper,doxy_lower,
#                                                                                            ds_AWG.lat,ds_AWG.lon,
#                                                                                            bounds,binwidth)

# # do smoothed version too


# # add to dataset
# ds_AWG['min_OCD_anom'] = xr.DataArray(min_OCD_anom,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_anom_month'] = xr.DataArray(min_OCD_anom_month,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_anom_TCD_anom'] = xr.DataArray(min_OCD_anom_TCD_anom,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_anom_doxy_upper'] = xr.DataArray(min_OCD_anom_doxy_upper,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])
# ds_AWG['min_OCD_anom_doxy_lower'] = xr.DataArray(min_OCD_anom_doxy_lower,
#                                    dims = ['latbins','lonbins'],coords =[latbins,lonbins])


In [ ]:
# # Plotting
# xx,yy = np.meshgrid(ds_AWG.lonbins,ds_AWG.latbins)
# xx = xx.flatten()
# yy = yy.flatten()

# bounds = [35,120,-20,30]
# s = 20

# fig, axes = plt.subplots(nrows = 2, ncols = 2,figsize=(20,12),subplot_kw={'projection': ccrs.PlateCarree()})

# # limits
# cmin = -25
# cmax = 25
# cmap = plt.cm.coolwarm
# p = axes[0,0].scatter(xx,yy,c = ds_AWG.min_OCD_anom.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[0,0],bounds)
# axes[0,0].set_extent(bounds, crs=ccrs.PlateCarree())
# axes[0,0].set_title('Minimum OCD Anomaly')
# cbar = fig.colorbar(p,ax=axes[0,0], pad=0.04)
# cbar.set_label('Pressure ($dbar$)')

# # limits
# cmin = 1
# cmax = 12
# cmap = plt.cm.Spectral
# p2 = axes[0,1].scatter(xx,yy,c = ds_AWG.min_OCD_anom_month.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[0,1],bounds)
# axes[0,1].set_title('Month of Minimum OCD Anomaly')
# cbar = fig.colorbar(p2,ax=axes[0,1], pad=0.04)
# cbar.set_label('month')

# # limits
# cmin = 0
# cmax = 200
# cmap = plt.cm.Spectral
# p3 = axes[1,0].scatter(xx,yy,c = ds_AWG.min_OCD_anom_doxy_upper.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[1,0],bounds)
# axes[1,0].set_extent(bounds, crs=ccrs.PlateCarree())
# axes[1,0].set_title('[O2] 0-50 dbar')
# cbar = fig.colorbar(p3,ax=axes[1,0], pad=0.04)
# cbar.set_label('Dissolved Oxygen ($units$)')

# cmin = 0
# cmax = 200
# cmap = plt.cm.Spectral
# p4 = axes[1,1].scatter(xx,yy,c = ds_AWG.min_OCD_anom_doxy_lower.stack(z=('latbins','lonbins')),marker='s',
#                       cmap=cmap,vmin=cmin,vmax=cmax,transform=ccrs.PlateCarree())
# add_land(axes[1,1],bounds)
# axes[1,1].set_title('[O2] 50-200 dbar')
# cbar = fig.colorbar(p4,ax=axes[1,1], pad=0.04)
# cbar.set_label('Dissolved Oxygen ($units$)')

In [ ]:
# this is where you left off
# tomorrow check the few places there is no woa data but there is merged data...that is plot the woa fields and make sure
# you were supposed to get something there. It might be that it couldnt find a TCD in the region...check it out.

# make the month of minimum oxygen plot and record the OCD and TCD for that value...

# figure out why there is negative oxygen data - remove it? 

# do that as a coastal map

# you should eventually tie this to the IOD index....maybe you can make maps that include only data from positive IOD years
# and negative etc...you should see some nice patterns there

# do the same for the Month of min oxygen...posID years it should be higher since you left the thermocline in BOB..

#you can make a coastl hovmuller here..first use the anomaly to find your coastlines since they are farther offshel, 
# then make the TCD OCD ones along with SLA...should show a nice patter for pos iod.neg iod years I hope


# add bincount maps


# save datasets



In [ ]:
np.array(ds_AWG.doxy_min)